# Perkins Core Indicators - Clean & Merge Files
Developed by Qingai "Sunny" Xu, CTE Research Expert, San Diego Community College District

This notebook provides a step-by-step guide to cleaning and merging data files for the Perkins Core Indicators Project. The cleaned data can then be uploaded to a Power BI dashboard for visualization and analysis.

Before running this notebook, make sure you have downloaded all the required files.  
To automate the data collection process, please visit [https://github.com/sunnyxu0628/PerkinsReportScraper](https://github.com/sunnyxu0628/PerkinsReportScraper), where I have developed code to streamline the downloading process.  

If you prefer to manually download the files, please use this notebook to clean the data:  
[**CleanPerkins-ManualDownloaded.ipynb**](CleanPerkins-ManualDownloaded.ipynb)


## Table of Contents

1. [Setup and Configuration](#setup)  
2. [Data Transformation and Combination](#data-transformation)  
3. [Data Cleaning](#data-cleaning)  
4. [Data Export](#data-export)  
5. [Optional: Data Reshaping (Long to Wide Format)](#reshaping)  


<a id='setup'></a>
## 1. Setup and Configuration


### Install Required Packages

Uncomment and run the cell below if you haven't installed the required packages. This should be done only once.

In [1]:
# %pip install pandas numpy os 
# %pip install seaborn

### Import Packages

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns

# Set pandas to display all rows
pd.set_option('display.max_rows', None)

### Configuration

Define your working environment and specify the number of colleges you're dealing with. This section allows you to easily adjust the notebook based on your specific needs.

In [27]:
# Configuration

import os
from pathlib import Path

# Base folder path for the Perkins project
Perkins_folder_path = Path('G:/Shared/SS/From RPNet/CTE Research/Perkins/Core Indicators/PerkinsReportScraper-main')

# Specify the list of colleges and their corresponding data paths
# You can add or remove entries from this dictionary based on the colleges you are working with


colleges = {
    # Format:
    # 'Standardized College Name': (Path to Top Code data folder, College name as it appears in the raw files)
    # The 'Standardized College Name' will be used in the final output files

    'City College': (Perkins_folder_path / 'Data/Top Code/San Diego City College', 'San Diego City College'),
    'Mesa College': (Perkins_folder_path / 'Data/Top Code/San Diego Mesa College', 'San Diego Mesa College'),
    'Miramar College': (Perkins_folder_path / 'Data/Top Code/San Diego Miramar College Reg Cntr', 'San Diego Miramar College Reg Cntr'),
    # Add more colleges here if needed
}

# Paths to college level and district level Core Indicators Data
college_level_path = Perkins_folder_path / 'Data/College/'
district_level_path = Perkins_folder_path / 'Data/District/'

# Paths to store raw and cleaned data
data_raw_path = Perkins_folder_path / 'Data'
data_clean_path = Perkins_folder_path / 'Data_Clean'
os.makedirs(data_clean_path, exist_ok=True)

# Export file name configuration
final_top6_export_file_name = 'Perkins_top6_2020_2026'
final_college_export_file_name = 'Perkins_college_2020_2026'

---

<a id='data-transformation'></a>
## 2. Data Transformation and Merging


### Create Lists of File Paths

In [28]:
# Create a dictionary to store file lists for each college
college_files = {}
  
# Create a dictionary to store file lists for each college
college_files = {}

for college_name, (path, _) in colleges.items():
    college_files[college_name] = [os.path.join(path, i) for i in os.listdir(path) if i.endswith('.csv')]

In [29]:
# Create a combined list of all college-level and district-level files
college_district_files = [college_level_path / i for i in os.listdir(college_level_path) if i.endswith('.csv')] + \
            [district_level_path / i for i in os.listdir(district_level_path) if i.endswith('.csv')]

### Helper functions: Define Functions for Data Transformation

In [30]:
def save_combine_tables(file_list, name):
    """
    Combine transformed tables and save as a single CSV file.

    Args:
        file_list (list): List of file paths.
        name (str): Name for the output CSV file.
    """
    # Initialize an empty list to store DataFrames
    dataframes = []

    # Iterate through each file path in the list
    for file_path in file_list:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Extract 'College' and 'Top Code' from the file name
        file_name = os.path.basename(file_path)
        college_name = file_name.split('_')[0]  # Extract the college name
        top_code = file_name.split('_')[2].replace('.csv', '')  # Extract the Top Code

        # Add 'College' and 'Top Code' columns to the DataFrame
        df.insert(0, 'College', college_name)  # Add 'College' as the first column
        df.insert(1, 'Top Code', top_code)    # Add 'Top Code' as the second column

        # Drop rows where 'Negotiated Level - District' is 'District'
        df = df[df['Negotiated Level - District'] != 'District']

        # Append the cleaned DataFrame to the list
        dataframes.append(df)

    # Combine all DataFrames into a single DataFrame
    output = pd.concat(dataframes, ignore_index=True)

    # Save the combined DataFrame to a CSV file
    output.to_csv(os.path.join(data_clean_path, name + '_combined_raw_top6.csv'), index=False)

In [31]:
def combine_college_district_tables(file_list):
    """
    Combine transformed tables into a single DataFrame.

    Args:
        file_list (list): List of file paths.

    Returns:
        pd.DataFrame: Combined DataFrame with added 'College/District' column.
    """
    # Initialize an empty list to store DataFrames
    dataframes = []

    # Iterate through each file path in the list
    for file_path in file_list:

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Extract 'College' from the file name
        file_name = os.path.basename(file_path)
        college_name = file_name.split('_')[0]  # Extract the college name

        # Adjust 'College' name: Keep "San Diego District" as is, otherwise drop "San Diego"
        if college_name == "San Diego District":
            pass
        else:
            college_name = college_name.replace("San Diego ", "")

        # Add 'College/District' column to the DataFrame
        df.insert(0, 'College/District', college_name)

        # Drop rows where 'Negotiated Level - District' is invalid
        df = df[df['Negotiated Level - District'] != 'Page 1 of 1']

        # Append the cleaned DataFrame to the list
        dataframes.append(df)
    output = pd.concat(dataframes, ignore_index=True)
    return output

### Apply Transformation and Save Combined Data

In [32]:
# Process and save the combined tables for each college
for college_name, files in college_files.items():
    save_combine_tables(files, college_name.replace(' ', ''))

In [33]:
college_district_df = combine_college_district_tables(college_district_files)

---

<a id='data-cleaning'></a>
## 3. Data Cleaning

### Top6 Level Files

#### Load Combined Data Files

In [34]:
# Load the combined CSV files into DataFrames
college_dfs = {}

for college_name in colleges.keys():
    df = pd.read_csv(os.path.join(data_clean_path, college_name.replace(' ', '') + '_combined_raw_top6.csv'), dtype=str)
    college_dfs[college_name] = df

#### Merge into a Single File & Rebale Columns

##### Helper Functions

In [35]:
#### Define if the core indicator meets the Negotiated Goal level
def calculate_90goal_flag(row):
    if row['Percent Above or Below 90% Negotiated Level'] == 'N/A':
        return 'N/A'
    elif row['Percent Above or Below 90% Negotiated Level'] == 'N/R':
        return 'N/R'
    elif float(row['Percent Above or Below 90% Negotiated Level']) >= 0 :
        return 'MET'
    elif float(row['Percent Above or Below 90% Negotiated Level']) < 0 :
        return 'NOT MET'

In [36]:
#### Define if the core indicator meets the 'District Negotiated Goal'
def calculate_district_goal(row):
    if row['Percent Above or Below Negotiated Level'] == 'N/A':
        return 'N/A'
    elif row['Percent Above or Below Negotiated Level'] == 'N/R':
        return 'N/R'
    elif pd.isna(row['College Performance Number']):
        return row['College Performance']
    elif float(row['Percent Above or Below Negotiated Level']) >= 0 :
        return 'MET'
    elif float(row['Percent Above or Below Negotiated Level']) < 0 :
        return 'NOT MET'

In [37]:
def transform_dataframe(df):
    """
    Transform a DataFrame by cleaning columns, and adding calculated fields.

    Args:
        df (pd.DataFrame): The input DataFrame to transform.

    Returns:
        pd.DataFrame: The transformed DataFrame.
    """

    # Split 'Core Indicator' into 'Core Indicator' and 'Cohort Year'
    df[['Core Indicator', 'Cohort Year']] = df['CI Number'].str.split(" - Cohort Yr: ", expand=True)
    df['Core Indicator'] = df['Core Indicator'].str.strip()
    df['Cohort Year'] = df['Cohort Year'].str.strip()

    # Fill missing values
    df['Percent Above or Below Negotiated Level'] = df['Percent Above or Below Negotiated Level'].fillna('N/A')
    df['Percent Above or Below 90% Negotiated Level'] = df['Percent Above or Below 90% Negotiated Level'].fillna('N/A')

    # Convert 'College Performance' to numeric
    df['College Performance Number'] = pd.to_numeric(df['College Performance'], errors='coerce')

    # Add calculated columns
    df['90% Negotiated Goal Indicator'] = df.apply(calculate_90goal_flag, axis=1)
    df['District Negotiated Goal Indicator'] = df.apply(calculate_district_goal, axis=1)
    df['Negotiated level - 90% District'] = (
        pd.to_numeric(df['Negotiated Level - District'], errors='coerce') * 0.9
    ).round(2)

    # Create additional columns
    df['Core Indicators'] = df['Core Indicator'].astype(str) + ' : ' + df['CI Info'].astype(str)
    df['College Performance Decimal'] = df['College Performance Number'] / 100
    df['Negotiated level - 90% District Decimal'] = pd.to_numeric(df['Negotiated level - 90% District'], errors='coerce') / 100
    df['Negotiated Level - State Decimal'] = pd.to_numeric(df['Negotiated Level - State'], errors='coerce') / 100
    df['Negotiated Level - District Decimal'] = pd.to_numeric(df['Negotiated Level - District'], errors='coerce') / 100
    df['Percent Above or Below Negotiated Level (%)'] = pd.to_numeric(
        df['Percent Above or Below Negotiated Level'], errors='coerce'
    )
    df['Percent Above or Below 90% Negotiated Level (%)'] = pd.to_numeric(
        df['Percent Above or Below 90% Negotiated Level'], errors='coerce'
    )

    # Calculate fiscal year
    def calculate_fiscal_year(cohort_year):
        # Extract the start year and calculate fiscal year
        start_year = int(cohort_year.split('-')[0]) + 3
        end_year = start_year + 1
        return f"{start_year} - {end_year}"

    df['Fiscal Year'] = df['Cohort Year'].apply(calculate_fiscal_year)

    return df

In [38]:
# Merge all DataFrames into a single DataFrame
all_df = pd.concat(college_dfs.values(), ignore_index=True)

#To ensure consistency, we'll standardize the college names in the 'College' column based on the configuration.

# Create a replacement dictionary from the colleges configuration
replacement_dict = {raw_name: standard_name for standard_name, (_, raw_name) in colleges.items()}

# Replace values in 'College' column to standardize names
all_df['College'] = all_df['College'].replace(replacement_dict)

### Clean Data Columns
# Clean 'Top Code' column
all_df['Top Code'] = all_df['Top Code'].str.replace('   ', '-')

all_df = transform_dataframe(all_df)

In [39]:
college_district_df = transform_dataframe(college_district_df)

---

<a id='data-export'></a>
## 6. Data Export

### Export Cleaned Data for Each College

In [40]:
# Export cleaned data for each college
for college_name in colleges.keys():
    college_df = all_df[all_df['College'] == college_name]
    college_df.to_csv(os.path.join(data_clean_path, college_name.replace(' ', '') + 'top6_clean.csv'), index=False)

### Save Combined Data

In [42]:
all_df.to_csv(os.path.join(data_clean_path, final_top6_export_file_name + '.csv'), index=False)
college_district_df.to_csv(os.path.join(data_clean_path, final_college_export_file_name + '.csv'), index=False)